Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give editor access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [1]:
## DONT CHANGE THIS CELL 
# this is currently the same as dev.data.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt

--2021-10-25 18:59:09--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt [following]
--2021-10-25 18:59:10--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63639 (62K) [text/plain]
Saving to: ‘test.data.txt’

test.data.txt       100%[===================>]  62.15K  64.9KB/s    in 1.0s    

2021-10-25 18:59:12 (64.9 KB/s) - ‘test.data.txt’ saved [63639/63639]



In [4]:
## Replace with the right link that contains the zip file uploaded from the training
!gdown https://drive.google.com/uc?id=1--UMT8OoxHmSZBks-N-l69lbQWnpsefp

Downloading...
From: https://drive.google.com/uc?id=1--UMT8OoxHmSZBks-N-l69lbQWnpsefp
To: /content/2018ME10032_B_model.zip
100% 1.07G/1.07G [00:07<00:00, 134MB/s]


In [5]:
!unzip 2018ME10032_B_model.zip

Archive:  2018ME10032_B_model.zip
  inflating: model.txt               


In [6]:
## Import relevant packages
import os
import re
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [7]:
!pip install transformers   # for hugging face libraries

     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 55.9 MB/s 
     |████████████████████████████████| 3.3 MB 33.4 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
## Limiting randomness
torch.manual_seed(0)
np.random.seed(0)

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Creating .CSV Test Data

In [10]:
def csv_creator(path_data='/content/drive/MyDrive/COL772_A3_B/data/train/train.data.txt', path_gold='/content/drive/MyDrive/COL772_A3_B/data/train/train.gold.txt', path_dir='content/', is_label=False, kind='train'):
    ## Reading Data File
    train_list = []
    count = 1
    with open(path_data, 'r') as f:
        for line in f:
            line = line.rstrip('\n')
            line = line.split('\t')
            token = line[0]
            entity = line[1]
            pos = line[2].split('-')
            pos1 = pos[0]
            pos2 = pos[1]
            sent1 = line[3]
            ## Punctuation if removed shall change the current order of words 
            ## and thus the significance of the indices given shall be lost
            sent1 = ' '.join(sent1.split())
            sent2 = line[4]
            sent2 = ' '.join(sent2.split())
            train_list.append({"Token": token, "Entity": entity, "Position1": pos1, 
                              "Position2": pos2, "Sentence1": sent1, "Sentence2": sent2})
        f.close()

    ## Creating DataFrame for the data
    train_df = pd.DataFrame(train_list)

    ## Label Encoding
    train_df['Entity'] = np.where(train_df['Entity'] == 'V', 1, 0)
    
    ## Adding position 2 relative to sentence 1 origin
    rel_pos = []
    for i in range(train_df.shape[0]):
        rel_pos.append(1 + len(list(train_df.iloc[i]['Sentence1'].split(' '))) + 1 + int(train_df.iloc[i]['Position2']))
    train_df['Position2_Rel'] = rel_pos

    # Reading Gold Labels (if there)
    if is_label:
        label_list = []
        with open(path_gold, 'r') as f:
            label_list = f.read().splitlines()
            f.close()

        ## Label Encoding
        for i in range(len(label_list)):
            if label_list[i] == 'T':
                label_list[i] = 1
            else:
                label_list[i] = 0
        train_df['Label'] = label_list
    
    ## Saving Data File
    path_csv = path_dir + str(kind) + '.csv'
    train_df.to_csv(path_csv)

In [11]:
csv_creator(path_data='/content/test.data.txt', path_dir = '/content/', is_label=False, kind='test')

In [12]:
test_data = pd.read_csv('/content/test.csv', index_col=0)

In [13]:
# test_data.head()

## Tokenizing

In [14]:
from transformers import BertTokenizer

In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [18]:
def dataset_creator(df):
    ## MAX LENGTH HAS BEEN REDUCED FOR FASTER TRAINING
    tokenized_data = tokenizer(list(df['Sentence1']), list(df['Sentence2']), 
                               max_length = 70, padding='max_length', truncation=True)
    return tokenized_data

In [19]:
test_tokens = dataset_creator(test_data)

## Creating Dataset

In [20]:
class WMCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, original_pos1, original_pos2):
        self.encodings = encodings
        # self.labels = labels
        self.pos1 = original_pos1
        self.pos2 = original_pos2
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # item['labels'] = torch.tensor(self.labels[idx])
        item['pos1'] = torch.tensor(self.pos1[idx])
        item['pos2'] = torch.tensor(self.pos2[idx])
        return item

    def __len__(self):
        return len(self.pos1)

In [21]:
test_dataset = WMCDataset(test_tokens, test_data['Position1'], test_data['Position2_Rel'])

In [22]:
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [23]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

## Creating Model

In [24]:
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained('bert-base-cased')
model.to(device)
model.train()

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [25]:
## Optimizer and Criterion for evaluating loss
optim = AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CosineEmbeddingLoss()          ## to gain a measure of similarity between the two word embeddings

In [26]:
## Checkpointing: Storing model state
def save_ckp(path, epoch, model, opt, loss):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'loss': loss,
            }, path)

In [27]:
seed_val = 42

np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

## Loading saved model 

In [28]:
## Loading saved model
checkpoint = torch.load('model.txt', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
optim.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [29]:
# print("model = ", model)
# print("optimizer = ", optim)
print("epoch = ", epoch)
print("loss = ", loss)

epoch =  4
loss =  0.31569499988108873


In [30]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## Defininng Loss & Accuracy Functions

In [31]:
## Defining accuracy measurements
def accuracy(preds, y):
    acc = torch.sum(preds == y) / y.shape[0]
    return acc

## Function to Calculate Loss
def calculate_loss(last_hidden_state, positions1, positions2):

    ## Picking out the specific word embeddings for out special token form last_hidden_state
    j = 1
    first = last_hidden_state[0, positions1[0], :]
    for i in positions1[1:]:
        first = torch.vstack((first, last_hidden_state[j, i, :]))
        j += 1

    ## Picking out the specific word embeddings for out special token form last_hidden_state
    j = 1
    second = last_hidden_state[0, positions2[0], :]
    for i in positions2[1:]:
        second = torch.vstack((second, last_hidden_state[j, i, :]))
        j += 1

    ## Finding cosine similarity between the two embeddings
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    res = cos(first, second).float()
    
    for i in range(len(res)):
        if res[i] > 0.7:      
          res[i] = 1
        else:
          res[i] = 0

    return res

In [32]:
# ## Generating results
# count = 0
# with torch.no_grad():
#     for batch in test_iterator:
#         if count == 0:
#             val_results = calculateLoss(model, batch, criterion)
#             count += 1
#         else:
#             val_results = torch.cat((val_results, calculateLoss(model, batch, criterion)), 0)
#     print(val_results)

In [33]:
## Generating Results
count = 0
with torch.no_grad(): 
    for batch in test_loader:
      optim.zero_grad()
      input_ids = batch['input_ids'].to(device)
      seg_ids = batch['token_type_ids'].to(device)
      attention_mask_ids = batch['attention_mask'].to(device)
      # labels = batch['labels'].to(device)
      pos1 = batch['pos1'].to(device)
      pos2 = batch['pos2'].to(device)

      output = model( input_ids, 
                      token_type_ids=seg_ids, 
                      attention_mask=attention_mask_ids, 
                      output_hidden_states=True
                    )
      
      last_hidden_state = output.hidden_states[-1].to(device)
      
      if count == 0:
          results = calculate_loss(last_hidden_state, pos1, pos2)
          count += 1
      else:
          results = torch.cat((results, calculate_loss(last_hidden_state, pos1, pos2)), 0)
    print(results)

tensor([0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
        1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1.,
        1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0.,
        1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
        1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0.,
        1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
        1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0.,
        0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
        0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
        1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1.,
        0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 1., 0., 

In [34]:
## Converting results to T and F format
res = []
for i in results:
    if i.int() == 0:
        res.append('F')
    else:
        res.append('T')
print(len(res))
print(res)

638
['F', 'F', 'F', 'T', 'T', 'F', 'T', 'T', 'T', 'F', 'F', 'F', 'T', 'F', 'F', 'F', 'T', 'F', 'T', 'T', 'T', 'T', 'T', 'F', 'F', 'T', 'T', 'T', 'F', 'F', 'T', 'T', 'F', 'T', 'F', 'T', 'T', 'T', 'T', 'F', 'T', 'F', 'F', 'T', 'F', 'F', 'T', 'T', 'F', 'T', 'F', 'T', 'T', 'F', 'T', 'T', 'T', 'F', 'T', 'F', 'T', 'T', 'T', 'F', 'T', 'T', 'F', 'T', 'T', 'T', 'T', 'T', 'T', 'F', 'T', 'T', 'F', 'T', 'F', 'T', 'F', 'F', 'F', 'F', 'T', 'T', 'T', 'F', 'T', 'T', 'F', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'F', 'F', 'F', 'T', 'F', 'T', 'T', 'T', 'F', 'T', 'T', 'T', 'T', 'F', 'F', 'T', 'F', 'T', 'T', 'T', 'T', 'F', 'T', 'T', 'T', 'F', 'F', 'T', 'T', 'T', 'T', 'T', 'F', 'T', 'F', 'F', 'F', 'T', 'T', 'T', 'T', 'T', 'T', 'F', 'F', 'F', 'T', 'T', 'T', 'F', 'T', 'T', 'F', 'F', 'T', 'T', 'T', 'T', 'T', 'T', 'F', 'F', 'T', 'F', 'T', 'F', 'F', 'F', 'T', 'T', 'T', 'F', 'T', 'T', 'F', 'F', 'T', 'F', 'T', 'T', 'T', 'T', 'F', 'F', 'T', 'F', 'T', 'F', 'F', 'F', 'F', 'T', 'F', 'T', 'F', 'T', 'F', 'F', 'T', 'F', 

In [35]:
## Saving results in the given format
with open('output.txt', 'w') as f:
    for item in res:
        f.write("%s\n" % item)

## Getting accuracy on test set

In [36]:
## DONT CHANGE THIS CELL
# Your testing code must produce a file output.txt with predictions as T and F in each line

## Final Evaluation 
# this is currently the same as dev.gold.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
correct, total = 0., 0.
for prediction, gold in zip(open('output.txt'), open('test.gold.txt')):
  prediction, gold = prediction.strip(), gold.strip()
  total += 1
  if prediction == gold:
    correct += 1

## Report this as the final validation performance 
print('Performance = ', (correct/total))

--2021-10-25 19:04:44--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt [following]
--2021-10-25 19:04:45--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1276 (1.2K) [text/plain]
Saving to: ‘test.gold.txt’

test.gold.txt       100%[===================>]   1.25K  --.-KB/s    in 0s      

2021-10-25 19:04:46 (42.2 MB/s) - ‘test.gold.txt’ saved [1276/1276]

Performance =  0.7084639498432602
